# Byte-Pair Encoding (BPE) 알아보기
> BPE, WordPiece, Unigram, SentencePiece

- toc: true
- badges: true
- comments: true
- date: 2022-07-07
- last-modified-at: 2022-07-07
- categories: [word embedding, BPE, wordpiece, unigram, sentencepiece, subword]

**Subword Tokenization**

Subword Tokenization이란 띄어쓰기(space)를 기준으로 나뉘는 단어보다 작지만 character(자/모)보다 큰 유닛(subword)으로 문장을 나누는 것으로 다음과 같은 장점이 있다. 
1. 빈도가 낮은 단어, 사전에 없는 단어들도 (빈도가 높은) 서브 단어들의 조합으로 인코딩할 수 있다.
2. 따라서 적당한 크기의 사전으로 많은 단어를 커버할 수 있다.
3. 띄어쓰기를 안하는 언어(예. 중국어, 일본어) 처리에 용이하다.
4. 접사/어미 등이 실질적 의미를 갖는 어근, 어간에 달라 붙어 어절을 형성하는 교착어인 한국어 처리에 보다 용이하다.

> note: 형태소 분석을 해서 하나의 형태소를 하나의 subword로 취급하는 것이 아니다. 

BPE의 다양한 알고리즘을 [huggingface의 설명](https://huggingface.co/docs/transformers/tokenizer_summary#bytepair-encoding-bpe)을 참고해 정리해봤다.

## 기본 아이디어

**훈련 방식**

0. 정규화
1. pre-tokenization & 기본 사전 만들기: 문장을 단어 단위로 나누기. 띄어쓰기 위주의 토큰화(e.g. GPT-2, Roberta 경우)도 가능하지만 보다 복잡한 토큰화를 사용할 수도 있다(GPT, XLM 등). 얻어진 토큰들을 기본 사전으로 삼고, 각 토큰들의 빈도수를 센다.
2. 위에서 구한 기본 사전들의 각 유닛들의 조합들 중 가장 빈도가 높은 조합을 사전에 추가한다.
3. 사전에 정한 크기에 사전이 될 때까지 (2)를 반복한다. 

**예시**

(1) 사용하는 corpus가 pre-tokenization 이후 다음과 같은 토큰들과 빈도수를 갖는다고 가정할 때
```
frquency = [("hug", 10), ("pug", 5), ("pun", 12), ("bun", 4), ("hugs", 5)]
vocabulary = ['b, 'g', 'h', 'n', 'p', 's', 'u']
>> [("h" "u" "g", 10), ("p" "u" "g", 5), ("p" "u" "n", 12), ("b" "u" "n", 4), ("h" "u" "g" "s", 5)]
```
(2) 사전에 등록된 아이템 조합 중 'u'+'g'가 가장 빈도가 높으므로 'ug'를 사전에 추가한다. 
```
("h" "ug", 10), ("p" "ug", 5), ("p" "u" "n", 12), ("b" "u" "n", 4), ("h" "ug" "s", 5)
>> vocabulary = ['b, 'g', 'h', 'n', 'p', 's', 'u', 'ug'] # update
```
(3) 정해진 사전 크기까지 '조합 + 사전 등록' 반복. 

> note: `</w>`와 같은 특별 토큰을 단어 끝에 붙여 단어간 경계를 표기하고 훈련시키기도 한다.

## Byte-level BPE 

Unicode가 아닌 Byte로 표현해 사전을 구성하기도 한다. 예를 들어 GPT-2는 바이트 기반으로 기본 사전을 만들기 때문에 256(==2^8)이라는 작은 크기의 기본 사전으로 1) 모든 영문자, 숫자와 일부 특수문자를 커버하고 2) 이들을 결합해 만든 50,000개의 조합과 <END-OF-TEXT>이라는 스페셜 토큰을 추가해 총 50,257짜리 사전을 구성한다.

**문제점**
빈도수가 똑같은 서브워드 쌍(pair)들이 여러 있을 때 어떤 쌍을 우선시할지 애매하다. 추가되는 쌍에 따라 같은 단어가 여러가지 방법으로 다르게 인코딩 될 수 있으며, 이는 최종 성능 평가에 영향을 줄 수 있다.

## WordPiece

BPE와 기본 아이디어와 훈련 방식이 거의 비슷한 알고리즘이 여러 있다. BERT등이 사용하는 WordPiece도 그 중 하나인데, 사전에 추가하는 기준이 살짝 다르다. BPE는 단순 빈도로 평가했다면, WordPiece는 가능도(likelihood)를 따진다. 또 위 BPE는 바이트들을 결합시켜 새 유닛을 만드는 bottom-up 방식인 반면, WordPiece는 bottom-up은 물론 top-down 방식으로도 구현할 수 있다(한국어, 일본어, 중국어 등은 top-down은 안됨).

WordPiece를 소개한 [논문](https://static.googleusercontent.com/media/research.google.com/ja//pubs/archive/37842.pdf)도 이를 설명하는 블로그들도 정확히 어떻게 가능도를 따지는지 설명하거나 코드를 제시하지 않는다. 논문에서 "Choose the new word unit out of all possible ones that increases the likelihood on the training data the most when added to the model" 이라고 하니, unigram 모델의 maximum likelihood를 따지는 것 같다.

즉 training corpus가 `(hug, 10), (pug, 5), (pun, 12)`라는 단어-카운트 쌍으로 이뤄져 있다고 가정하고, 가능한 조합쌍 'pu'와 'ug' 중 하나만 사전에 넣어야 한다면

In [46]:
from collections import Counter
import math

corpus = ['hug'] * 10 + ['pug'] * 5 + ['pun'] * 12
counter = Counter(''.join(corpus))
counts = counter.most_common()
print(f'basic vocabulary: {counts}')
print('After 1st possible iteration')
for cand in ['pu', 'ug']:
    new_counter = counter.copy()
    for word in corpus:
        if cand in word:
            new_counter.update({cand:word.count(cand)}) 
            for char in cand:
                new_counter.update({char:-1})
    counts = new_counter.most_common() 
    print(f'possible candidate: {cand} & updated vocabulary: {counts}')   
    values = [v for (k,v) in counts if v > 0]
    model_likelihood = math.prod(values) / sum(values)**len(values)
    print(f'possible candidate: {cand} & likelihood: {model_likelihood}')


basic vocabulary: [('u', 27), ('p', 17), ('g', 15), ('n', 12), ('h', 10)]
After 1st possible iteration
possible candidate: pu & updated vocabulary: [('pu', 17), ('g', 15), ('n', 12), ('h', 10), ('u', 10), ('p', 0)]
possible candidate: pu & likelihood: 0.0002849847078323364
possible candidate: ug & updated vocabulary: [('p', 17), ('ug', 15), ('u', 12), ('n', 12), ('h', 10), ('g', 0)]
possible candidate: ug & likelihood: 0.0002932128470001566


단순 frequency를 따지는 BPE로 하면 'pu'(17)를 'ug'(15) 대신 사전에 등록해야 하지만, WordPiece는 모델의 가능도가 더 높은 'ug'를 추가한다. 물론 이는 bottom-up 알고리즘이며, space를 표기하는 space marker, 즉 _(underscore)를 고려하지 않은 예시이다.
그런데 BERT에 사용되었다는 top-down WordPiece 알고리즘은 BPE처럼 가능도 대신 단순 빈도를 따지고 subword unit 후보도 독특하게 구성하는 듯 하다. [여기](https://www.tensorflow.org/text/guide/subwords_tokenizer#optional_the_algorithm) 참고.


https://pytorch.org/tutorials/beginner/chatbot_tutorial.html
https://wikidocs.net/22592

## Reference
1. huggingface: [BPE Tokenizer Summary](https://huggingface.co/docs/transformers/tokenizer_summary#bytepair-encoding-bpe)
2. huggingface: [BPE Tokenization](https://huggingface.co/course/chapter6/5?fw=pt#bytepair-encoding-tokenization)
3. google-tensorflow: [Subword Tokenization](https://www.tensorflow.org/text/guide/subwords_tokenizer#optional_the_algorithm)